In [ ]:
from dotenv import load_dotenv
load_dotenv('.env', override=True)

In [ ]:
from langsmith import evaluate, Client
from rag import ask, initialize

client = Client()
dataset_name = "glimmerbooms"

initialize('eval_samples') # local dir name containing documents

def is_concise(outputs: str, reference_outputs: dict) -> bool:
    # Verify that the length of the model's output does not exceed 1.5 times the length of the reference output.
    if len(outputs['output']) < 1.5 * len(reference_outputs['output']):
        return True
    return False

def target(input):
    res = ask(input['input'])
    return {'output': res} # format expected by the evaluator

evaluate(
    target,
    data=client.list_examples(dataset_name=dataset_name, split=['factual']),
    evaluators=[is_concise],
    experiment_prefix="exp with rep",
    num_repetitions=1,
    metadata={
        "model_name": 'gemini'
    }
)